In [3]:
import scanpy as sc
import numpy as np
from cell_load.utils.data_utils import filter_on_target_knockdown
orig_path="vcc_data/adata_Training.h5ad"
data_path="vcc_data/arc/perturb_processed.h5ad"
adata = sc.read_h5ad(orig_path)
sc.pp.log1p(adata)
adata.obs["condition"] = adata.obs["target_gene"]
adata.obs["condition"] = (
    adata.obs["target_gene"].replace("non-targeting","ctrl")
)
adata.var["gene_name"] = adata.var.index.values
print(adata.shape)
# adata = filter_on_target_knockdown(
#     adata=adata,
#     perturbation_column="condition",           # Column in obs containing perturbation info
#     control_label="ctrl",                 # Label for control cells
#     residual_expression=0.30,             # Perturbation-level threshold (30% residual = 70% knockdown)
#     cell_residual_expression=0.50,        # Cell-level threshold (50% residual = 50% knockdown)
#     min_cells=30,                         # Minimum cells per perturbation after filtering
#     layer=None,                           # Use adata.X (or specify a layer)
#     var_gene_name="gene_name"             # Column in var containing gene names
# )   
# print(f"Shape of adata after filtering {adata.shape}")  

adata.obs["condition"]=adata.obs['condition'].cat.rename_categories({cat:f"ctrl+{cat}" for cat in adata.obs.condition.cat.categories if cat != "ctrl"})
adata.obs["cell_type"] = "h1_ESC"
adata.obs["dose_val"] = np.where(adata.obs["condition"] == "ctrl", "1", "1+1")
adata.obs["condition_name"] = adata.obs["cell_type"].astype(str)+"_"+ adata.obs["condition"].astype(str) + "_" + adata.obs["dose_val"].astype(str)
 
sc.tl.rank_genes_groups(adata, groupby="condition", n_genes=5000, method="wilcoxon")
adata.uns["rank_genes_groups_cov_all"]  = adata.uns["rank_genes_groups"]
adata.write_h5ad(filename=data_path)
print(adata)
print(adata.obs)


(221273, 18080)
AnnData object with n_obs × n_vars = 221273 × 18080
    obs: 'target_gene', 'guide_id', 'batch', 'condition', 'cell_type', 'dose_val', 'condition_name'
    var: 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts', 'mt', 'ribo', 'n_cells', 'gene_name'
    uns: 'log1p', 'rank_genes_groups', 'rank_genes_groups_cov_all'
                                      target_gene  \
AAACAAGCAACCTTGTACTTTAGG-Flex_1_01          CHMP3   
AAACAAGCATTGCCGCACTTTAGG-Flex_1_01           AKT2   
AAACCAATCAATGTTCACTTTAGG-Flex_1_01          SHPRH   
AAACCAATCCCTCGCTACTTTAGG-Flex_1_01         TMSB4X   
AAACCAATCTAAATCCACTTTAGG-Flex_1_01          KLF10   
...                                           ...   
TTTGGACGTGGTGCAGATTCGGTT-Flex_3_16  non-targeting   
TTTGTGAGTAGTAGCAATTCGGTT-Flex_3_16          KDM1A   
TTTGTGAGTCCATCCTATTCGGTT-Flex_3_16  non-targeting   
TTTGTGAGTCCTGACAATTCGGTT-Flex_3_16          BIRC2   
TTTGTGAGTGGACACG

In [6]:
adata.write_h5ad("de_.h5ad")

In [7]:
adata.uns

OrderedDict([('log1p', {'base': None}),
             ('rank_genes_groups',
              {'params': {'groupby': 'condition',
                'reference': 'rest',
                'method': 'wilcoxon',
                'use_raw': False,
                'layer': None,
                'corr_method': 'benjamini-hochberg'},
               'names': rec.array([('HMGCS1', 'PKDCC', 'EPHB3', 'CD24', 'PLAAT3', 'H1F0', 'SOCS3', 'MMP24', 'CASP3', 'CHGA', 'ADGRG1', 'OPN3', 'DNMT3B', 'PODXL', 'MFGE8', 'ASS1', 'PSAT1', 'GAL', 'ADCY3', 'VIM', 'HCFC1R1', 'TSTA3', 'AASS', 'PHF21B', 'ACSS2', 'RYBP', 'PTBP2', 'PCK2', 'DPPA5', 'STAR', 'ERBB2', 'IFRD1', 'RASGRF2', 'STC2', 'S100A11', 'SERPINB9', 'ACSS2', 'ZNF664', 'STC2', 'COL1A1', 'MT-ND3', 'PCK2', 'VIM', 'PLAAT3', 'TMEM65', 'CD24', 'HMGCS1', 'PGRMC1', 'MEX3B', 'NODAL', 'VIM', 'MT-ND3', 'CHD3', 'SFRP1', 'H2AFJ', 'ALDH6A1', 'CD24', 'STC2', 'MT-ND4', 'RAC3', 'GAL', 'GNPDA1', 'EGR1', 'SPTSSB', 'PCK2', 'OPN3', 'FOXO4', 'CAMK2D', 'ID3', 'CHD7', 'YTHDF2', 'FZD2', 'S

In [ ]:
adata.write_h5ad(filename="filtered_adata_Training.h5ad")

In [2]:
import scanpy as sc
data_path="/scratch/saigum/PerturbationPredictionUsingGeneNetworks/vcc_data/arc/perturb_processed.h5ad"
adata = sc.read_h5ad(data_path)
import pickle as pk
with open("/scratch/saigum/PerturbationPredictionUsingGeneNetworks/vcc_data/gene2go_all.pkl","rb") as f:
    datago = pk.load(f)

In [3]:

key_set = set(datago.keys())
genes = set(adata.var_names)         # O(n)
common  = genes & key_set            # O(min(n, m))
count   = len(common)

print(count)


17525


In [5]:
import scanpy as sc 
test = sc.read_h5ad("/scratch/saigum/PerturbationPredictionUsingGeneNetworks/demo/data/norman/perturb_processed.h5ad")
print(test.obs)

                        condition cell_type dose_val  control  \
cell_barcode                                                    
AAACCTGAGGCATGTG-1   TSC22D1+ctrl      A549      1+1        0   
AAACCTGAGGCCCTTG-1    KLF1+MAP2K6      A549      1+1        0   
AAACCTGCACGAAGCA-1           ctrl      A549        1        1   
AAACCTGCAGACGTAG-1  CEBPE+RUNX1T1      A549      1+1        0   
AAACCTGCAGCCTTGG-1     MAML2+ctrl      A549      1+1        0   
...                           ...       ...      ...      ...   
TTTGTCAGTCATGCAT-8   RHOXF2BB+SET      A549      1+1        0   
TTTGTCATCAGTACGT-8     FOXA3+ctrl      A549      1+1        0   
TTTGTCATCCACTCCA-8     CELF2+ctrl      A549      1+1        0   
TTTGTCATCCCAACGG-8    BCORL1+ctrl      A549      1+1        0   
TTTGTCATCTGGCGAC-8    MAP4K3+ctrl      A549      1+1        0   

                            condition_name  
cell_barcode                                
AAACCTGAGGCATGTG-1   A549_TSC22D1+ctrl_1+1  
AAACCTGAGGCCCTTG-1 

In [7]:
test.obs["condition_name_2"] = test.obs["cell_type"].astype(str)+"_"+ test.obs["condition"].astype(str) + "_" + test.obs["dose_val"].astype(str) 


In [11]:
(test.obs["condition_name_2"] == test.obs["condition_name"]).sum()

np.int64(91205)

In [2]:
test.var

,gene_name
gene_id,
ENSG00000239945,RP11-34P13.8
ENSG00000223764,RP11-54O7.3
ENSG00000187634,SAMD11
ENSG00000187642,PERM1
ENSG00000188290,HES4
...,...
ENSG00000198786,MT-ND5
ENSG00000198695,MT-ND6
ENSG00000198727,MT-CYB


In [ ]:
from gears import PertData
pert_data = PertData('./vcc_data') # specific saved folder
pert_data.load(data_path="/scratch/saigum/PerturbationPredictionUsingGeneNetworks/vcc_data/arc/")


In [ ]:
from gears import GEARS
pert_data.prepare_split(split = 'simulation', seed = 1)
pert_data.get_dataloader(batch_size = 32, test_batch_size = 128)
gears_model = GEARS(pert_data, device = 'cuda:7', 
                        weight_bias_track = False, 
                        proj_name = 'pertnet', 
                        exp_name = 'pertnet')
gears_model.model_initialize(hidden_size = 64)
gears_model.train(epochs = 20, lr = 1e-3)

In [ ]:
pert_data.load(data_path="/scratch/saigum/PerturbationPredictionUsingGeneNetworks/vcc_data/arc/")


In [ ]:
pert_data.prepare_split(split = 'simulation', seed = 1) # get data split with seed
pert_data.get_dataloader(batch_size = 32, test_batch_size = 128) # prepare data loader

In [ ]:
import scanpy as sc
adata_train = sc.read_h5ad("/scratch/saigum/PerturbationPredictionUsingGeneNetworks/vcc_data/arc_train/perturb_processed.h5ad")
print(adata_train)